# Skript zum Testen der Objekterkennung

In [ ]:
!pip install opencv-python ultralytics

In [ ]:
model_path = "path/to/gesture_yolov8n.pt" # <-- ggf. Pfad anpassen
# e.g.
#model_path = "/workspaces/ai_tutorial/ros_ws/src/ai_tutorial/model/gesture_yolov8n.pt"

In [ ]:
import cv2
from ultralytics import YOLO

# Load your trained YOLOv8 model (PyTorch .pt file)
model = YOLO(model_path)  # Update the path

# Set class names if not embedded in the model
# default dataset
class_names = ["down", "left", "no_gesture", "right", "stop", "up"]

# dataset with the thumbs_up gesture
#class_names = ["down", "left", "no_gesture", "right", "stop", "thumbs_up" ,"up"]

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference
    results = model(frame, imgsz=640, conf=0.4)[0]  # results[0] = first image in batch

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        label = class_names[cls_id] if cls_id < len(class_names) else f"ID:{cls_id}"

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("YOLOv8n Detection (.pt)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Skript zur Aufnahme von Bildern

In [ ]:
import cv2
import time
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Zielordner
output_dir = "thumbs_up"
fps = 5
interval = 1 / fps
os.makedirs(output_dir, exist_ok=True)

# Kamera öffnen (0 = erste Kamera, 1 = zweite, ...)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Kann die Kamera nicht öffnen")
    cap.release()

frame_count = 0
last_capture_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Bild nur alle "interval"-Sekunden speichern
        current_time = time.time()
        if current_time - last_capture_time >= interval:
            filename = os.path.join(output_dir, f"frame_{frame_count:05d}.jpg")
            cv2.imwrite(filename, frame)
            print(f"Gespeichert: {filename}")
            frame_count += 1
            last_capture_time = current_time

        # Aktuelles Bild anzeigen (inline)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        clear_output(wait=True)
        plt.imshow(frame_rgb)
        plt.axis('off')
        plt.show()

except KeyboardInterrupt:
    print("Aufnahme abgebrochen.")

cap.release()


## Aufteilung in Trainings- und Validierungsdaten

In [ ]:
import os
import random
from pathlib import Path
import shutil

def split_dataset(input_images_dir, input_labels_dir, output_base_dir, val_ratio=0.2, seed=42):
    random.seed(seed)

    # Alle Bilder im Eingabeverzeichnis finden
    images = list(Path(input_images_dir).glob("*.jpg")) + list(Path(input_images_dir).glob("*.png"))
    if not images:
        raise ValueError("Keine Bilddateien im Eingabeverzeichnis gefunden.")

    # Zufällig mischen und aufteilen
    random.shuffle(images)
    val_count = int(len(images) * val_ratio)
    val_images = images[:val_count]
    train_images = images[val_count:]

    def copy_files(image_list, split):
        image_out_dir = Path(output_base_dir) / split / "images"
        label_out_dir = Path(output_base_dir) / split / "labels"
        image_out_dir.mkdir(parents=True, exist_ok=True)
        label_out_dir.mkdir(parents=True, exist_ok=True)

        for image_path in image_list:
            label_path = Path(input_labels_dir) / (image_path.stem + ".txt")
            shutil.copy(image_path, image_out_dir / image_path.name)
            if label_path.exists():
                shutil.copy(label_path, label_out_dir / label_path.name)

    # Aufteilen und kopieren
    copy_files(train_images, "train")
    copy_files(val_images, "val")

    print(f"Datensatz aufgeteilt: {len(train_images)} Trainingsbilder, {len(val_images)} Validierungsbilder.")

# Beispielaufruf – passe die Pfade bei Bedarf an
split_dataset(
    input_images_dir="thumbs_up",           # Pfad zu deinen Bildern
    input_labels_dir="thumbs_up",           # Pfad zu deinen Labels
    output_base_dir="dataset",              # Ausgabeordner
    val_ratio=0.2                           # 20 % für Validierung
)